In [4]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
import pandas as pd
import os
import warnings
import glob

warnings.filterwarnings("ignore")
pd.set_option("display.max_columns", 6)
pd.set_option("display.max_rows", 6)
np.random.seed(2)

## Load the data and process it

In [8]:
dataset_path = "dataset/"

def load_all_scenes():
    scene_ids = [file.split("/")[-1].split(".")[0] for file in glob.glob(f"{dataset_path}*.nodes")]
    all_scenes = []
    for scene_id in scene_ids:
        edges_file = os.path.join(dataset_path, f"{scene_id}.edges")
        nodes_file = os.path.join(dataset_path, f"{scene_id}.nodes")

        edges_df = pd.read_csv(edges_file, header=None, names=["target", "source"], na_values="_")
        nodes_df = pd.read_csv(
            nodes_file,
            header=None,
            names=["node_id", "current_x", "current_y", "previous_x", "previous_y", "future_x", "future_y"],
            na_values="_"
        )

        if nodes_df.isnull().any().any():
            nan_nodes = nodes_df[nodes_df.isnull().any(axis=1)]
            nan_node_ids = nan_nodes["node_id"].tolist()
            edges_df = edges_df[~edges_df["source"].isin(nan_node_ids) & ~edges_df["target"].isin(nan_node_ids)]
            nodes_df = nodes_df.dropna(subset=["future_x", "future_y"])

        if (edges_df["source"] == -1).any() or (edges_df["target"] == -1).any():
            edges_df = edges_df[(edges_df["source"] != -1) & (edges_df["target"] != -1)]
            connected_nodes = pd.unique(edges_df[["target", "source"]].values.ravel())
            nodes_df = nodes_df[nodes_df["node_id"].isin(connected_nodes)]

        nodes_df = nodes_df.replace('_', np.nan)
        edges_df = edges_df.replace('_', np.nan)

        if len(nodes_df) > 0:
            node_id_to_idx = {node_id: idx for idx, node_id in enumerate(nodes_df["node_id"].values)}
            edges_df = edges_df.dropna()
            edges_df['target'] = edges_df['target'].apply(lambda x: node_id_to_idx.get(x, -1))
            edges_df['source'] = edges_df['source'].apply(lambda x: node_id_to_idx.get(x, -1))
            edges_df = edges_df[(edges_df['target'] != -1) & (edges_df['source'] != -1)]

            all_scenes.append({
                "scene_id": scene_id,
                "node_features": tf.convert_to_tensor(
                    nodes_df[["current_x", "current_y", "previous_x", "previous_y"]].values,
                    dtype=tf.float32
                ),
                "labels": tf.convert_to_tensor(nodes_df[["future_x", "future_y"]].values, dtype=tf.float32),
                "edges": tf.convert_to_tensor(edges_df[["target", "source"]].values, dtype=tf.int32),
                "node_indices": np.arange(len(nodes_df)),
            })
            
    return all_scenes


scenes = load_all_scenes()

# Shuffle and split into train, validation, and test sets
np.random.seed(2)
np.random.shuffle(scenes)

""" 70% Training set """
# train_scenes = scenes[:int(0.7 * len(scenes))]
# val_scenes = scenes[int(0.7 * len(scenes)):int(0.9 * len(scenes))]
# test_scenes = scenes[int(0.9 * len(scenes)):]

""" 50% Training set """
train_scenes = scenes[:int(0.5 * len(scenes))]
val_scenes = scenes[int(0.5 * len(scenes)):int(0.9 * len(scenes))]
test_scenes = scenes[int(0.9 * len(scenes)):]

print(f"Total scenes: {len(scenes)}")
print(f"Training scenes: {len(train_scenes)}")
print(f"Validation scenes: {len(val_scenes)}")
print(f"Test scenes: {len(test_scenes)}")

def scene_to_sample(scene):
    return (scene["node_features"], scene["edges"], scene["labels"])

train_samples = [scene_to_sample(scene) for scene in train_scenes]
val_samples = [scene_to_sample(scene) for scene in val_scenes]
test_samples = [scene_to_sample(scene) for scene in test_scenes]

output_signature = (
    tf.TensorSpec(shape=(None, 4), dtype=tf.float32),
    tf.TensorSpec(shape=(None, 2), dtype=tf.int32),
    tf.TensorSpec(shape=(None, 2), dtype=tf.float32),
)

train_dataset = tf.data.Dataset.from_generator(
    lambda: iter(train_samples),
    output_signature=output_signature
)

val_dataset = tf.data.Dataset.from_generator(
    lambda: iter(val_samples),
    output_signature=output_signature
)

test_dataset = tf.data.Dataset.from_generator(
    lambda: iter(test_samples),
    output_signature=output_signature
)

def squeeze_batch(features, edges, targets):
    return tf.squeeze(features, axis=0), tf.squeeze(edges, axis=0), tf.squeeze(targets, axis=0)

train_dataset = train_dataset.shuffle(100).batch(1).map(squeeze_batch)
val_dataset = val_dataset.batch(1).map(squeeze_batch)
test_dataset = test_dataset.batch(1).map(squeeze_batch)


Total scenes: 189
Training scenes: 94
Validation scenes: 76
Test scenes: 19


## GAT model implementation

In [6]:
class GraphAttention(layers.Layer):
    def __init__(
        self,
        units,
        kernel_initializer="glorot_uniform",
        kernel_regularizer=None,
        **kwargs,
    ):
        super().__init__(**kwargs)
        self.units = units
        self.kernel_initializer = keras.initializers.get(kernel_initializer)
        self.kernel_regularizer = keras.regularizers.get(kernel_regularizer)

    def build(self, input_shape):
        self.kernel = self.add_weight(
            shape=(input_shape[0][-1], self.units),
            trainable=True,
            initializer=self.kernel_initializer,
            regularizer=self.kernel_regularizer,
            name="kernel",
        )
        self.kernel_attention = self.add_weight(
            shape=(self.units * 2, 1),
            trainable=True,
            initializer=self.kernel_initializer,
            regularizer=self.kernel_regularizer,
            name="kernel_attention",
        )
        self.built = True

    def call(self, inputs):
        node_states, edges = inputs

        # Linearly transform node states
        node_states_transformed = tf.matmul(node_states, self.kernel)

        # (1) Compute pair-wise attention scores
        node_states_expanded = tf.gather(node_states_transformed, edges)
        node_states_expanded = tf.reshape(
            node_states_expanded, (tf.shape(edges)[0], -1)
        )
        attention_scores = tf.nn.leaky_relu(
            tf.matmul(node_states_expanded, self.kernel_attention)
        )
        attention_scores = tf.squeeze(attention_scores, -1)

        # (2) Normalize attention scores
        attention_scores = tf.math.exp(tf.clip_by_value(attention_scores, -2, 2))
        attention_scores_sum = tf.math.unsorted_segment_sum(
            data=attention_scores,
            segment_ids=edges[:, 0],
            num_segments=tf.reduce_max(edges[:, 0]) + 1,
        )
        attention_scores_sum = tf.repeat(
            attention_scores_sum, tf.math.bincount(tf.cast(edges[:, 0], "int32"))
        )
        attention_scores_norm = attention_scores / attention_scores_sum

        # (3) Gather node states of neighbors, apply attention scores and aggregate
        node_states_neighbors = tf.gather(node_states_transformed, edges[:, 1])
        out = tf.math.unsorted_segment_sum(
            data=node_states_neighbors * attention_scores_norm[:, tf.newaxis],
            segment_ids=edges[:, 0],
            num_segments=tf.shape(node_states)[0],
        )
        return out


class MultiHeadGraphAttention(layers.Layer):
    def __init__(self, units, num_heads=8, merge_type="concat", **kwargs):
        super().__init__(**kwargs)
        self.num_heads = num_heads
        self.merge_type = merge_type
        self.attention_layers = [GraphAttention(units) for _ in range(num_heads)]

    def call(self, inputs):
        atom_features, pair_indices = inputs

        # Obtain outputs from each attention head
        outputs = [
            attention_layer([atom_features, pair_indices])
            for attention_layer in self.attention_layers
        ]
        # Concatenate or average the node states from each head
        if self.merge_type == "concat":
            outputs = tf.concat(outputs, axis=-1)
        else:
            outputs = tf.reduce_mean(tf.stack(outputs, axis=-1), axis=-1)
        # Activate and return node states
        return tf.nn.relu(outputs)


class GraphAttentionNetwork(keras.Model):
    def __init__(
        self,
        #node_states,
        #edges,
        hidden_units,
        num_heads,
        num_layers,
        output_dim,
        **kwargs,
    ):
        super().__init__(**kwargs)
        #self.node_states = node_states
        #self.edges = edges
        self.preprocess = layers.Dense(hidden_units * num_heads, activation="relu")
        self.attention_layers = [
            MultiHeadGraphAttention(hidden_units, num_heads) for _ in range(num_layers)
        ]
        self.output_layer = layers.Dense(output_dim)

    def call(self, inputs):
        node_states, edges = inputs
        x = self.preprocess(node_states)
        for attention_layer in self.attention_layers:
            x = attention_layer([x, edges]) + x
        outputs = self.output_layer(x)
        return outputs

    def train_step(self, data):
        node_features, edges, labels = data  # Unpack the batch
        with tf.GradientTape() as tape:
            outputs = self([node_features, edges], training=True)
            loss = self.compiled_loss(labels, outputs)
        grads = tape.gradient(loss, self.trainable_weights)
        self.optimizer.apply_gradients(zip(grads, self.trainable_weights))
        self.compiled_metrics.update_state(labels, outputs)
        return {m.name: m.result() for m in self.metrics}

    def predict_step(self, data):
        node_features, edges, _ = data  # labels may not be present
        outputs = self([node_features, edges], training=False)
        return outputs

    def test_step(self, data):
        node_features, edges, labels = data
        outputs = self([node_features, edges], training=False)
        loss = self.compiled_loss(labels, outputs)
        self.compiled_metrics.update_state(labels, outputs)
        return {m.name: m.result() for m in self.metrics}


# Task 1 - adjust the model to the new data

In [ ]:
# Define hyper-parameters
HIDDEN_UNITS = 100
NUM_HEADS = 8
NUM_LAYERS = 3
OUTPUT_DIM = 2

NUM_EPOCHS = 100
LEARNING_RATE = 1e-4

# Build model
gat_model = GraphAttentionNetwork(
    HIDDEN_UNITS, NUM_HEADS, NUM_LAYERS, OUTPUT_DIM
)

callbacks = keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

# Compile model
gat_model.compile(
    loss=keras.losses.MeanAbsoluteError(name="mean_absolute_error"),
    optimizer=keras.optimizers.Adam(
        learning_rate=LEARNING_RATE,
        clipnorm=1.0, 
        epsilon=1e-8
    ),
    metrics=[keras.metrics.MeanAbsoluteError(name="mean_absolute_error")]
)

gat_model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=NUM_EPOCHS,
    verbose=1,
    callback=callbacks
)

_, test_accuracy = gat_model.evaluate(x=test_dataset, verbose=1)
print(f"Test Mean Absolute Error: {test_accuracy}")


all_predictions = []
all_ground_truth = []
sample_count = 0

print("Evaluating model on entire test dataset")

for features, edges, targets in test_dataset:
    predictions = gat_model((features, edges), training=False)
    
    # Convert to numpy for easier handling
    pred_np = predictions.numpy()
    targets_np = targets.numpy()
    
    # Store predictions
    all_predictions.append(pred_np)
    all_ground_truth.append(targets_np)
    
    sample_count += len(pred_np)

print(f"Processed {len(all_predictions)} scenes with {sample_count} total nodes")

# Concatenate all predictions and ground truth values
all_pred_concat = np.concatenate(all_predictions)
all_gt_concat = np.concatenate(all_ground_truth)

# Calculate MAE for x and y coordinates separately
mae_x = np.mean(np.abs(all_pred_concat[:, 0] - all_gt_concat[:, 0]))
mae_y = np.mean(np.abs(all_pred_concat[:, 1] - all_gt_concat[:, 1]))

# Calculate overall MAE
mae = np.mean(np.abs(all_pred_concat - all_gt_concat))

# Calculate Euclidean distance for each prediction
euclidean_distances = np.sqrt(np.sum((all_pred_concat - all_gt_concat)**2, axis=1))
mean_euclidean = np.mean(euclidean_distances)
median_euclidean = np.median(euclidean_distances)

print("\n===== Evaluation Metrics =====")
print(f"Mean Absolute Error (overall): {mae:.2f}")
print(f"Mean Absolute Error (x-coordinate): {mae_x:.2f}")
print(f"Mean Absolute Error (y-coordinate): {mae_y:.2f}")
print(f"Mean Euclidean Distance: {mean_euclidean:.2f} units")
print(f"Median Euclidean Distance: {median_euclidean:.2f} units")


Epoch 1/25
     93/Unknown 24s 14ms/step - mean_absolute_error: 9752.5205 - loss: 4095.2783

2025-04-18 12:54:20.246728: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]


94/94 ━━━━━━━━━━━━━━━━━━━━ 28s 52ms/step - mean_absolute_error: 9679.8311 - loss: 4096.7524 - val_loss: 5486.8877
Epoch 2/25
 9/94 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - mean_absolute_error: 4981.3716 - loss: 6003.3071

2025-04-18 12:54:23.773949: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]


93/94 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - mean_absolute_error: 4659.2939 - loss: 4735.2754

...

2025-04-18 12:54:56.764904: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]


94/94 ━━━━━━━━━━━━━━━━━━━━ 2s 16ms/step - mean_absolute_error: 965.4800 - loss: 5281.7705 - val_loss: 6902.9746
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - mean_absolute_error: 632.5240 - loss: 5475.7734


2025-04-18 12:54:57.028976: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
2025-04-18 12:54:57.138363: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]


Test Mean Absolute Error: {'mean_absolute_error': <tf.Tensor: shape=(), dtype=float32, numpy=633.5399>}
Evaluating model on entire test dataset
Processed 19 scenes with 170 total nodes

===== Evaluation Metrics =====
Mean Absolute Error (overall): 633.54
Mean Absolute Error (x-coordinate): 715.36
Mean Absolute Error (y-coordinate): 551.72
Mean Euclidean Distance: 1020.98 units
Median Euclidean Distance: 868.19 units


2025-04-18 12:55:01.307498: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
